# Notebook to send commands to a list of devices
This notebook sends commands to a list of devices. nornir is used to speed up the process.

___To use this notebook...___
1. configure your username and password
2. set where statement to get the list of devices
3. configure token to access the sot
4. If you want to see some more DEBUG messages set loglevel to DEBUG (optional)

### define your login here

In [ ]:
username = "username"
password = "password"

### on which devices should the commands be executed

In [ ]:
where = ""
additional_data = {}

### configure our SOT

In [ ]:
token = "your_token"
url = "http://127.0.0.1:8080"
ssl_verify = False

### set loglevel

In [ ]:
loglevel = "CRITICAL"
scrapli_loglevel = "NONE"

# Basic settup.. Import packages and add functions

In [ ]:
import pandas as pd
import sys
from loguru import logger
from veritas.sot import sot as veritas_sot
from nornir_inspect import nornir_inspect
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result
from nornir_napalm.plugins.tasks import napalm_get
from nornir_scrapli.tasks import send_configs
from nornir_netmiko.tasks import netmiko_save_config, netmiko_send_config

In [ ]:
logger.configure(extra={"extra": "unset"})
logger.remove()
logger.add(sys.stderr, level=loglevel)

## function to initialize nornir

In [ ]:
def init_nornir():
    sot = veritas_sot.Sot(token=token, url=url, ssl_verify=ssl_verify)
    nr = sot.job.on(where) \
        .set(username=username, password=password, result='result', parse=False) \
        .add_data(additional_data) \
        .init_nornir()
    return nr

## function to send commands to device

In [ ]:
def send_commands_to_device(task: Task, commands) -> Result:
    result = []
    # Manually create Netmiko connection
    net_connect = task.host.get_connection("netmiko", task.nornir.config)
    result.append({'cmd': 'config_mode', 'output': net_connect.config_mode()})
    for cmd in commands:
        result.append({'cmd': cmd, 'output': net_connect.send_command(cmd, expect_string=r"#")})
    result.append({'cmd': 'exit_config_mode', 'output': net_connect.exit_config_mode()})
    return Result(
        host=task.host,
        result=result
    )

# now the fun part begins

## initialize nornir

In [ ]:
nr = init_nornir()

## check the list of hosts

In [ ]:
display(nr.inventory.hosts)

## define your commands here

In [ ]:
commands = ["username xxx"]

## now run commands on all devices
at the end a table containing all results is printed

In [ ]:
results = nr.run(
            name="send_commands_to_device", 
            task=send_commands_to_device,
            commands=commands
)

In [ ]:
table = []
hosts = results.keys()
for host in hosts:
    commands = results[host][0].result
    for command in commands:
        key = command.get('cmd')
        value = command.get('output')
        table.append({'host': host, 'cmd': key, 'output': value})
df = pd.DataFrame(table)
df